In [ ]:
# Блок 1: Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import seaborn as sns
from tqdm import tqdm

# Настройки отображения
plt.style.use('ggplot')
pd.options.display.max_columns = 10

In [ ]:
# Блок 2: Функция для загрузки данных
def load_sensor_data(file_path):
    """Загружает данные из файла с обработкой метаданных и основного DataFrame"""
    # Чтение метаданных
    metadata = {}
    with open(file_path, 'r', encoding='cp1251') as f:
        for line in f:
            if line.startswith('Время,'):  # Конец метаданных
                break
            if ',' in line:
                key, val = line.strip().split(',', 1)
                metadata[key] = val
    
    # Чтение основного DataFrame
    df = pd.read_csv(file_path, 
                    encoding='cp1251',
                    skiprows=len(metadata)+1, 
                    na_values=['. . . . .', ' '],
                    low_memory=False)
    
    # Парсинг диапазонов значений
    ranges = df.iloc[0].str.split('_', expand=True).T
    ranges.columns = df.columns
    ranges = ranges.rename(index={0: 'min', 1: 'max'})
    
    df = df.iloc[1:].reset_index(drop=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    
    return metadata, df, ranges

In [ ]:
# Блок 3: Визуализация данных для одного файла
def plot_sensor_data(df, title=''):
    """Визуализация временных рядов"""
    plt.figure(figsize=(15, 20))
    for i, col in enumerate(df.columns[1:], 1):  # Пропускаем колонку времени
        plt.subplot(len(df.columns)-1, 1, i)
        plt.plot(df['Время'], df[col])
        plt.title(col)
        plt.grid(True)
    plt.suptitle(title, y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# Блок 4: Расчет метрик
def calculate_metrics(df, ranges):
    """Расчет основных метрик для анализа"""
    metrics = pd.DataFrame({
        'mean': df.mean(),
        'std': df.std(),
        'min': df.min(),
        'max': df.max(),
        'range_min': ranges.loc['min'].values,
        'range_max': ranges.loc['max'].values
    })
    
    # Проверка выхода за пределы диапазона
    metrics['out_of_range'] = (df.min() < metrics['range_min'].astype(float)) | \
                             (df.max() > metrics['range_max'].astype(float))
    
    return metrics

In [ ]:
# Блок 5: Обработка всех файлов
all_files = glob.glob('./data/**/*_аналоговые.csv', recursive=True)
all_metrics = []

for file in tqdm(all_files):
    # Загрузка данных
    metadata, df, ranges = load_sensor_data(file)
    
    # Визуализация (можно закомментировать для многих файлов)
    plot_sensor_data(df, title=os.path.basename(file))
    
    # Расчет метрик
    metrics = calculate_metrics(df, ranges)
    metrics['file'] = os.path.basename(file)
    all_metrics.append(metrics.reset_index())
    
# Объединение всех метрик
combined_metrics = pd.concat(all_metrics)

In [ ]:
# Блок 6: Анализ всех данных
print(f"Всего обработано файлов: {len(all_files)}")
print("\nПример метаданных:")
print(metadata)

# Сводная статистика по всем файлам
summary_stats = combined_metrics.groupby('index').agg({
    'mean': ['mean', 'std'],
    'std': ['mean', 'std'],
    'out_of_range': 'sum'
})

display(summary_stats)

# Визуализация распределений параметров
plt.figure(figsize=(15, 10))
sns.boxplot(data=combined_metrics, x='index', y='mean')
plt.xticks(rotation=90)
plt.title('Распределение средних значений по всем файлам')
plt.show()

In [ ]:
# Блок 7: Анализ корреляций (на примере последнего файла)
_, df, _ = load_sensor_data(all_files[0])
corr_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Матрица корреляций (пример для одного файла)')
plt.show()